# Load Libraries

In [1]:
import os
import sys
import math
import warnings
import logging
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import dotenv
import pyet


import matplotlib.pyplot as plt

from linearmodels.panel import PanelOLS, RandomEffects
from linearmodels.panel import compare

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from scipy.optimize import minimize
from scipy.stats import randint, uniform

import lightgbm as lgb
from lightgbm import LGBMRegressor

# Set warnings
warnings.filterwarnings("ignore")

# Load environment variables from .env file
dotenv.load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO)

# Suppress warnings
warnings.filterwarnings("ignore")

# Set display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_colwidth', None)

# Load Data

In [2]:
# Load Data From Pickle
with open('../../data/ETo/Khorasan_ETo_Data_1950_2025.pkl', 'rb') as f:
    data = pickle.load(f)
    logging.info("Data loaded from pickle file.")

INFO:root:Data loaded from pickle file.


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2263576 entries, 0 to 2263575
Data columns (total 36 columns):
 #   Column             Dtype         
---  ------             -----         
 0   date               datetime64[ns]
 1   station_id         object        
 2   station_name       object        
 3   region_id          object        
 4   region_name        object        
 5   lat                float64       
 6   lon                float64       
 7   station_elevation  float64       
 8   tmax               float64       
 9   tmin               float64       
 10  tm                 float64       
 11  umax               float64       
 12  umin               float64       
 13  um                 float64       
 14  ffm                float64       
 15  sshn               float64       
 16  rrr24              float64       
 17  0                  float64       
 18  Penman             float64       
 19  FAO-56             float64       
 20  Priestley-Taylor   float

In [4]:
# Filter Data
df = data[[
    "region_name",
    "region_id",
    "station_name",
    "station_id",
    "station_elevation",
    "lat",
    "lon",
    "date",
    "tmax",
    "tmin",
    "tm",
    "umax",
    "umin",
    "um",
    "ffm",
    "sshn",
    "rrr24",
    "FAO-56",
    "Penman",
    "Hargreaves",
]]

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2263576 entries, 0 to 2263575
Data columns (total 36 columns):
 #   Column             Dtype         
---  ------             -----         
 0   date               datetime64[ns]
 1   station_id         object        
 2   station_name       object        
 3   region_id          object        
 4   region_name        object        
 5   lat                float64       
 6   lon                float64       
 7   station_elevation  float64       
 8   tmax               float64       
 9   tmin               float64       
 10  tm                 float64       
 11  umax               float64       
 12  umin               float64       
 13  um                 float64       
 14  ffm                float64       
 15  sshn               float64       
 16  rrr24              float64       
 17  0                  float64       
 18  Penman             float64       
 19  FAO-56             float64       
 20  Priestley-Taylor   float

In [6]:
df[['region_name', 'station_name']].drop_duplicates().reset_index(drop=True)

,region_name,station_name
0,South Khorasan,Ghohestan
1,South Khorasan,Eshgh Abad
2,South Khorasan,Fath Abad Ferdows
3,South Khorasan,Ghani Abad
4,South Khorasan,Seghale
...,...,...
77,North Khorasan,Raz
78,North Khorasan,Maneh-Va-Samalqan
79,North Khorasan,Farouj
80,North Khorasan,Esfarayen


In [7]:
min_rows = 10 * 365
df_filtered = df.groupby(['region_id', 'station_id']).filter(lambda x: x['FAO-56'].count() >= min_rows).reset_index(drop=True)
logging.info(f"{df_filtered[['region_id', 'station_id']].nunique()}")

INFO:root:region_id      3
station_id    19
dtype: int64


In [8]:
df_filtered[['region_name', 'region_id', 'station_name', 'station_id']].drop_duplicates().reset_index(drop=True)

,region_name,region_id,station_name,station_id
0,South Khorasan,OIMB,Tabas,40791
1,South Khorasan,OIMB,Ferdows,40792
2,South Khorasan,OIMB,Qaen,40793
3,South Khorasan,OIMB,Birjand,40809
4,South Khorasan,OIMB,Nehbandan,40827
5,South Khorasan,OIMB,Boshruyeh,99407
6,Khorasan Razavi,OIMM,Quchan,40740
7,Khorasan Razavi,OIMM,Sarakhs,40741
8,Khorasan Razavi,OIMM,Sabzevar,40743
9,Khorasan Razavi,OIMM,Golmakan,40744


# Intra-station Modeling

In [9]:
predictors = ['Hargreaves', 'tm', 'tmin', 'tmax']
targets = ['FAO-56', 'Penman']

df_filtered.dropna(subset=predictors + targets, inplace=True)

X = df_filtered[predictors]
y = df_filtered[targets]

df_filtered['ml_pred_fao56'] = np.nan
df_filtered['ml_pred_penman'] = np.nan

station_models = {}

print("\n--- شروع مدل‌سازی درون‌ایستگاهی (ML) ---")
for station in df_filtered['station_id'].unique():
    print(f"آموزش مدل برای ایستگاه: {station}")

    df_station = df_filtered[df_filtered['station_id'] == station]
    X_station = df_station[predictors]
    y_station = df_station[targets]
    
    model = RandomForestRegressor(n_estimators=200, random_state=42)
    model.fit(X_station, y_station)
    
    station_models[station] = model
    
    predictions = model.predict(X_station)
    
    df_filtered.loc[df_filtered['station_id'] == station, 'ml_pred_fao56'] = predictions[:, 0]
    df_filtered.loc[df_filtered['station_id'] == station, 'ml_pred_penman'] = predictions[:, 1]

print("مدل‌سازی درون‌ایستگاهی با موفقیت انجام شد.")
print("نمونه‌ای از پیش‌بینی‌های مدل ML:")
print(df_filtered)


--- شروع مدل‌سازی درون‌ایستگاهی (ML) ---
آموزش مدل برای ایستگاه: 40791
آموزش مدل برای ایستگاه: 40792
آموزش مدل برای ایستگاه: 40793
آموزش مدل برای ایستگاه: 40809
آموزش مدل برای ایستگاه: 40827
آموزش مدل برای ایستگاه: 99407
آموزش مدل برای ایستگاه: 40740
آموزش مدل برای ایستگاه: 40741
آموزش مدل برای ایستگاه: 40743
آموزش مدل برای ایستگاه: 40744
آموزش مدل برای ایستگاه: 40745
آموزش مدل برای ایستگاه: 40746
آموزش مدل برای ایستگاه: 40762
آموزش مدل برای ایستگاه: 40763
آموزش مدل برای ایستگاه: 40778
آموزش مدل برای ایستگاه: 40806
آموزش مدل برای ایستگاه: 40723
آموزش مدل برای ایستگاه: 99287
آموزش مدل برای ایستگاه: 99295
مدل‌سازی درون‌ایستگاهی با موفقیت انجام شد.
نمونه‌ای از پیش‌بینی‌های مدل ML:
           region_name region_id station_name station_id  station_elevation  \
6926    South Khorasan      OIMB        Tabas      40791             711.00   
6927    South Khorasan      OIMB        Tabas      40791             711.00   
6928    South Khorasan      OIMB        Tabas      40791             711.00

# Inter-station Modeling

In [ ]:
df_panel = df_filtered.set_index(['station_id', 'date'])

df_filtered['panel_pred_fao56'] = np.nan
df_filtered['panel_pred_penman'] = np.nan

print("\n--- شروع مدل‌سازی بین‌ایستگاهی (Panel) ---")

for target_variable in targets:
    print(f"آموزش مدل پانلی برای متغیر: {target_variable}")
    
    formula = f"{target_variable} ~ 1 + {' + '.join(predictors)} + EntityEffects"
    model_panel = PanelOLS.from_formula(formula, data=df_panel)
    results = model_panel.fit()
    
    if target_variable == 'panel_pred_fao56':
        df_filtered[f'panel_pred_fao56'] = results.predict().values.flatten()
    if target_variable == 'panel_pred_penman':
        df_filtered[f'panel_pred_penman'] = results.predict().values.flatten()

print("مدل‌سازی بین‌ایستگاهی با موفقیت انجام شد.")
print("نمونه‌ای از پیش‌بینی‌های مدل Panel:")
print(df)